In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

2022-07-24 09:56:03.550795: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [54]:
size = 128
IMAGE_SIZE = [size, size]

In [55]:
train_path="/home/nayan/Documents/Thesis/dataset/new clean/train"
test_path= "/home/nayan/Documents/Thesis/dataset/new clean/test"
val_path=  "/home/nayan/Documents/Thesis/dataset/new clean/test"

In [56]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(size,size))
        x_train.append(img_arr)

/home/nayan/Documents/Thesis/dataset/new clean/train/Brown_Spot
/home/nayan/Documents/Thesis/dataset/new clean/train/Hispa
/home/nayan/Documents/Thesis/dataset/new clean/train/Leaf_Blast


In [57]:
x_test=[]

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(size,size))
        x_test.append(img_arr)

/home/nayan/Documents/Thesis/dataset/new clean/test/Brown_Spot
/home/nayan/Documents/Thesis/dataset/new clean/test/Hispa
/home/nayan/Documents/Thesis/dataset/new clean/test/Leaf_Blast


In [58]:
x_val=[]

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(size,size))
        x_val.append(img_arr)

In [59]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [60]:
train_x.shape,test_x.shape,val_x.shape


((441, 128, 128, 3), (76, 128, 128, 3), (76, 128, 128, 3))

In [61]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [62]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [63]:
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (size, size),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (size, size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (size, size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 441 images belonging to 3 classes.
Found 76 images belonging to 3 classes.
Found 76 images belonging to 3 classes.


In [64]:
training_set.class_indices


{'Brown_Spot': 0, 'Hispa': 1, 'Leaf_Blast': 2}

In [65]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
train_y.shape,test_y.shape,val_y.shape

((441,), (76,), (76,))

In [66]:
# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [67]:
for layer in vgg.layers:
    layer.trainable = False

In [68]:
x = Flatten()(vgg.output)

prediction = Dense(3, activation='softmax')(x)

In [69]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [70]:
# tell the model what cost and optimization method to use
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [72]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=30,  
  batch_size=32,
    shuffle=True)

Epoch 1/30
14/14 [==============================] - 4s 309ms/step - loss: 0.1413 - accuracy: 0.9773 - val_loss: 0.5487 - val_accuracy: 0.8421
Epoch 2/30
14/14 [==============================] - 5s 328ms/step - loss: 0.1442 - accuracy: 0.9796 - val_loss: 0.5679 - val_accuracy: 0.8289
Epoch 3/30
14/14 [==============================] - 4s 317ms/step - loss: 0.1377 - accuracy: 0.9841 - val_loss: 0.6227 - val_accuracy: 0.8026
Epoch 4/30
14/14 [==============================] - 4s 323ms/step - loss: 0.1362 - accuracy: 0.9773 - val_loss: 0.6132 - val_accuracy: 0.8158
Epoch 5/30
14/14 [==============================] - 5s 328ms/step - loss: 0.1243 - accuracy: 0.9819 - val_loss: 0.5616 - val_accuracy: 0.8553
Epoch 6/30
14/14 [==============================] - 4s 325ms/step - loss: 0.1165 - accuracy: 0.9819 - val_loss: 0.5663 - val_accuracy: 0.8553
Epoch 7/30
14/14 [==============================] - 4s 325ms/step - loss: 0.1135 - accuracy: 0.9819 - val_loss: 0.5061 - val_accuracy: 0.8684
Epoch 